# HPC in Julia
## Lecture I - π in many ways
### Recap of CPU solution

Yesterday, you computed approximations to π by Monte Carlo simulations by utilizing that if $(X_i)$ and $(Y_i)$ are independent uniformly distributed random variables on the unit interval then the Law of Large Numbers gives
$$
\frac{1}{n}\sum_{i=1}^n \mathbb{1}\{X_i^2 + Y_i^2 < 1\} \overset{P}{\longrightarrow} 
E \mathbb{1}\{X_i^2 + Y_i^2 < 1\} = \frac{\pi}{4}.
$$

In this notebook, we will use this result to introduce different modes of parallelism in Julia. First, we will redo the calculation on CPU and use that as a reference for comparison when converting the implementation to GPU, distributed and threaded versions.

The version we will use here is based on arrays and higher order functions such as map and reduce (here as a sum). As we will see by the end, this version might not always be the fastest possible but it will be competitive and it also has the advantage that it looks very similar to the math behind it.

In [1]:
simulate_π_cpu(n) = 4*sum(map((x, y) -> x^2 + y^2 < 1, rand(Float32, n), rand(Float32, n)))/n

simulate_π_cpu (generic function with 1 method)

Again, we'll load `BenchmarkTools` for the timings

In [2]:
using BenchmarkTools
@btime simulate_π_cpu(10)

  544.984 ns (9 allocations: 720 bytes)


3.6

For for `n = 10`, the CPU version is 300-400 times faster than the GPU version. Let's see the effecting using a ten times larger `n`.

In [3]:
@btime simulate_π_cpu(100)

  1.240 μs (9 allocations: 1.61 KiB)


3.36

The CPU version scales roughly linearly whereas the GPU version is roughly unaffected. This is explained by the large constant costs of memory operations on the GPU. Now lets try `n = 10^6`.

In [4]:
@btime simulate_π_cpu(10^6)

  7.409 ms (12 allocations: 8.58 MiB)


3.141324

In order to get a better idea about how this scales, we'll collect some more timings

In [5]:
ns = 3:23
cpu_timings = [minimum(Base.@elapsed simulate_π_cpu(2^n) for i in 1:5) for n in ns];

and define a small convenience function for ploting the timings

In [6]:
using PlotlyWebIO
function plot_timings(sz::AbstractVector, times::Vector...; names = Vector{String} = ["t$i" for i in 1:length(times)])
    n = length(times)
    
    if length(times) != n
        throw(DimensionMismatch(""))
    end

    ps = vcat(map(zip(times, names)) do tn
        scatter(
            x = 2.^sz,
            y = tn[1],
            name = tn[2]
        )
    end...)

    ps12 = hcat(
        Plot(ps,
            Layout(
                title = "Timings on linear scales"
            )
        ),
        Plot(ps,
            Layout(
                title = "Timings on logarithmic scales",
                xaxis_type = "log",
                yaxis_type = "log",
            )
        )
    )

    return PlotlyWebIO.WebIOPlot(ps12)
end

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/528251b1ff164bb91886525e074d4435e4f7eab2-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/528251b1ff164bb91886525e074d4435e4f7eab2-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

plot_timings (generic function with 1 method)

In [7]:
plot_timings(ns, cpu_timings, names = ["CPU"])

data: [
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, xaxis1, xaxis2, yaxis1, and yaxis2"

### GPUs in Julia

For this section to work, the notebook should be run on where a NVidia GPU is available. Typically, an easy way of querying for available GPUs (devices) is to run

In [8]:
;nvidia-smi

Tue Jul  3 02:46:41 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.59                 Driver Version: 390.59                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 23%   39C    P2    57W / 250W |   3744MiB / 11178MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:0E:00.0 Off |                  N/A |
| 30%   52C    P2    71W / 250W |  10664MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

This should show a table listing the available GPUs as well as a list of processes running on the GPU.

Julia is able to utilize NVidia GPUs through a set of packages of varying level of abstraction. For many usecases, the low level packages can be ignored but it is important to emphasize that the high level package are mostly built on top of the low level packages so if you want to dive into the details then you are in fact able to do so while not leaving Julia at all.

<p align="left|right">

 Package      | Description 
 :----------- | :------------:
 `CUDAdrv`    | Access the CUDA driver API from Julia 
 `CUDAnative` | Compile and execute native Julia kernels on CUDA hardware 
 `CuArrays`   | Fully-functional GPU arrays similar to normal Julia Arrays


The package `CUDAdrv` handles interactions with the GPU driver and provides some convenience functions e.g. to show the available devices

In [9]:
using CUDAdrv
map(name, devices())

1-element Array{String,1}:
 "Tesla P100-PCIE-16GB"

where `devices()` returns an iterator of device objects and `name` returns a string identifying the device.

The `CUDAnative` package provides functionality for compiling (a subset of) Julia to GPUs. When loading the package, it will automatically initialize a device with an associated *contect*. We can use the context to see which device has been assigned to us.

In [10]:
using CUDAnative
device(CuCurrentContext())

CuDevice(0): Tesla P100-PCIE-16GB

The package allows users to write GPU kernels but since GPU kernel writing is a demanding task we won't provide many details at this point but later hint at what is possible with `CUDAnative`.

Instead, we'll focus on the `CuArrays` package which allows users to utilize the power of GPU through higher order functions on arrays such as `map`/`broadcast` and `reduce`.

####  Example: $\pi$ on a GPU

To show the power of GPU programming in Julia with `CuArrays` we can revisit the example of computing $\pi$ as a Monte Carlo simulation.

It is possible to draw random numbers very fast in parallel on a GPU if we use a specialized library. Hence, instead of drawing the variates one-by-one, we'll draw two arrays of random numbers in the GPU version

In [11]:
using CuArrays
using CuArrays.CURAND
simulate_π_gpu(n) = 4*sum(map((x, y) -> x^2 + y^2 < 1, curand(Float32, n), curand(Float32, n)))/n

simulate_π_gpu (generic function with 1 method)

As usual, the function is compiled the first time it is called and GPU functions are quite slow to compile.

In [ ]:
@time simulate_π_gpu(10)

Again, we'll load `BenchmarkTools` for the timings

In [ ]:
using BenchmarkTools
@btime simulate_π_gpu(10)

For for `n = 10`, the CPU version is 300-400 times faster than the GPU version. Let's see the effecting using a ten times larger `n`.

In [ ]:
@btime simulate_π_gpu(100)

The CPU version scaled roughly linearly whereas the GPU version is roughly unaffected by the 10x increase in sample size. This is explained by the large constant costs of memory operations on the GPU. Now lets try `n = 10^6`.

In [ ]:
@btime simulate_π_gpu(10^6)

Again, we'll produce a vector of timings and plot together with the CPU timings for easy comparison

In [ ]:
gpu_timings = [minimum(Base.@elapsed simulate_π_gpu(2^n) for i in 1:5) for n in ns];
plot_timings(ns, cpu_timings, gpu_timings, names = ["CPU", "GPU"])

### Distributed Arrays

Julia has two different ways in which users can run Julia code interactively on multiple processes. The one we will use in this notebook is built into the core Julia distribution and should generally be chosen when you run Julia on your own hardware. Later, you using JuliaBox for exercises, you'll be introduced to another methods for using multiple processes. The two methods differ in how set up the processes but after the workers have been set up, most user facing functions are the same. The differences are monstly hidden to users but some of them are important to be aware of.

As an introduction to multiprocesses in Julia, we'll now run simulate yet again. This time using eight Julia worker processes. First, we add the workers and load the `DistributedArrays` package to get access to arrays that can work accross multiple processes.

In [ ]:
if nprocs() == 1
    addprocs(8)
end
using DistributedArrays

When loading package that should be available on all workers, it is important that these are loaded **after** the processes have been set up.

Next, we define the simulation function. Notice that it is almost identical to the previous versions. The only difference here is that the random numbers are generated the `drand` function instead of `rand` and `curand`. As a prefix to the function, we have added `@everywhere`. This is necessary when defining functions "at the global scope" or "top-level" but is not necessary if a function is defined locally, e.g. as is the case for the `(x, y) -> x^2 + y^2 < 1` anonymous function.

In [ ]:
@everywhere simulate_π_dist(n) = 4*sum(map((x, y) -> x^2 + y^2 < 1, drand(Float32, n), drand(Float32, n)))/n

Again, we can run a few benchmarks to get an idea about the performance

In [ ]:
@btime simulate_π_dist(10)
@btime simulate_π_dist(100)
@btime simulate_π_dist(10^6)

Similarly to the GPU version, the sample size appears to have little influence on the timings.

Again we plot the tinings together with the previous timings to get a better understanding of the scaling

In [ ]:
dist_timings = [minimum(Base.@elapsed simulate_π_dist(2^n) for i in 1:5) for n in ns];
plot_timings(ns, cpu_timings, gpu_timings, dist_timings,
    names = ["CPU", "GPU", "Distributed ($(nworkers()) CPUs)"])

From the plots, we can see that the distributed version scales as expected but that the constant overhead is even more pronounced for distributed arrays compated to the GPU version. The cross-over point is around `n=10^6`.

## Threading (experimental)

Julia has native threading. However, the API is only being settled this spring and summer. What is presented below will probably become obsolete within the next year but it works and was the founadation for the single node parallelism in Celeste Project.

Because the final API for Julia's threading hasn't been settled yet, threading hasn't been built into `map` and `broadcast` operations yet. We therefore cheat a little bit and define our custom threaded two-argument `tmap` to use in the comparison with the other implementations of the π simulator.

In [ ]:
# 1. Restricting elements to subtypes of AbstractFloat isn't strictly necessary
# but makes it easier to compute the return type of the map
# 2. Avoiding branches in function with the threaded for loop to help
# the compiler (currently it has a huge effect)
function _tmap!(f::F, y::Vector{T}, x1::Vector{T}, x2::Vector{T}) where {F<:Function, T<:AbstractFloat}
    n = length(y)
    
    Threads.@threads for i in 1:n
        y[i] = f(x1[i], x2[i])
    end
    
    return y
end

function tmap(f::F, x1::Vector{T}, x2::Vector{T}) where {F<:Function,T<:AbstractFloat}
    
    # Check input args
    n = length(x1)
    if n != length(x2)
        throw(DimensionMismatch("arrays must have the same length"))
    end
    
    # Allocate output array
    y = similar(x1)
    
    # Call the threaded kernel
    @inbounds _tmap!(f, y, x1, x2)
    
    return y
end

simulate_π_threads(n) = 4*sum(tmap((t, s) -> t^2 + s^2 < 1, rand(Float32, n), rand(Float32, n)))/n

In [ ]:
@btime simulate_π_threads(10)

In [ ]:
@btime simulate_π_threads(100)

In [ ]:
@btime simulate_π_threads(10^6)

In [ ]:
threads_timings = [minimum(Base.@elapsed simulate_π_threads(2^n) for i in 1:5) for n in ns];

In [ ]:
plot_timings(ns, cpu_timings, gpu_timings, dist_timings, threads_timings,
    names = ["CPU", "GPU", "Distributed ($(nworkers()) CPUs)", "Threaded ($(Threads.nthreads()) threads)"])

The speedup from utilizing threading is modest. This is not surprising since the process of generating random variables hasn't been parallelized.

## A faster CPU version

All the functions compared so far are based on allocating arrays of random numbers. By building up the computations from arrays, maps, and reductions, it is easy to utilize GPUs and distributed ressources. However, as we have seen, there is varying degrees of overhead associated with memory allocation.

The Monte Carlo simulation actually doesn't require allocation of any arrays and we will finish off this demo with a version the doesn't allocate any arrays.

In [ ]:
# Notice that rand() is faster than rand(Float32)
simulate_π_cpu_noalloc(n) = 4*sum(t -> rand()^2 + rand()^2 < 1, 1:n)/n

In [ ]:
cpu_noalloc_times = [minimum(Base.@elapsed simulate_π_cpu_noalloc(2^n) for i in 1:5) for n in ns];

In [ ]:
plot_timings(ns, cpu_timings, gpu_timings, dist_timings, threads_timings, cpu_noalloc_times,
    names = ["CPU", "GPU", "Distributed ($(nworkers()) CPUs)", "Threaded ($(Threads.nthreads()) threads)", "CPU noalloc"])

In [ ]:
using DataFrames
DataFrame(
    n = 2.^ns, 
    CPU_timings = cpu_timings,
    GPU_timings = gpu_timings,
    GPU_speedup = cpu_timings ./ gpu_timings,
    Distributed_timings = dist_timings,
    Distributed_speedup = cpu_timings ./ dist_timings,
    Threaded_timings = threads_timings,
    Threaded_timings = cpu_timings ./ threads_timings
)

In [32]:
Threads.nthreads()

8

In [33]:
Threads.threadid()

1